# John Trygier Optimization Project 

## Dually Maximizing Truck Fill Rate with Minimized Supply Chain Length

### 12/3/2021

I understand that this project report is optional  for  extra  credit,  that  the  amount  of  points 
awarded  by  this  submission  is  solely  determined  by  the  instructor  assessment,  and  that  no 
requests  for  regrading  or  complaints  about  the  number  of  extra  points  awarded  will  be 
admitted.

<html>
<head>
<style>
h1 {text-align: center;}
p {text-align: center;}
div {text-align: center;}
</style>
</head>
<body>

<h1>The Problem</h1>
    
    
    
    
    
    
    
    
    
    
    
    
    
    
<p>Achieving high truck fill rates (TFR) is an integral component of actively and effectively managing any supply chain.</p>

<p> 
    
</p>
<div>Poor fill rates directly translate into wasted time in terms of needing to make more trips in order to satisfy the same demand. Empty space in a given truck thus causes waste in fuel, labor hours, management, and lost confidence from the customer due to delays. It further undermines our ability to optimize a supply chain, or to exercise control over a given process. </div>

</body>
</html>

Even in a given situation in which we are not losing time by needing to make multiple trips to fulfill demand, we are still exposing a waste of overprocessing inherent to our supply chain. Rather than negotiating with customers and suppliers about lead times and delivery dates, we fulfill demand in suboptimal conditions, representing a suboptimal approach to meeting our customer demand and internal efficiency metrics.

Poor TFR's comes from many places, I personally saw them at Toyota during my internships when trying to maximize fill rates across the dock. The difficulty comes from how we manage the dual problem of 

1) minimizing the discrepancy between the size of the truck vs. the amount of necessary product to be shipped (which may not be symmetrically distributed), and

2) maximizing the utility of a given truck to a supply chain.


What I mean when I reference this "discrepancy" is that it will not always be possible to construct truckloads of material that can be arranged to fill the truck to optimality. We may not have sufficient demand necessary to load the truck to optimality, we may be filling orders for parts that are sized in an incongruous manner that will make it impossible to fill the truck when mixing a variety of items in one truck (as is often necessary). When attempting to maximize a truck's utility, I am referring to the benefit that the truck provides to the customer, the demand that it meets, as well as the benefit of cost saving metrics yielded by a minimized route. 

Once a truck has been filled and shipped, it will then need to be routed in order to serve the customer in the best way that we can find, which changes the remaining demand to be met, which changes the optimal solution, and so on. As our system of variables interact with one another, we must find a way to consider how to maximize our TFR, while routing the prepared truck my minimal route length.

## A Review of the Literature




In their paper studying the use of Network Optimization, Prakash, Chan, Liao, and Deshmukh propose that "most supply chain design problems can be reduced to a capacitated facility location problem (CFLP), which is known to be NP-Complete; therefore most supply chain problems are NP-hard" (Prakash et.al. 1), which indicates that the problem is exponentially difficult when reflecting optimality for the ground-truth scenario. They recommend the utilization of various relaxations of the base-case model to ensure computational complexity is reduced, including a Lagrangian relaxation of the problem, but they also present the idea of "multi-objective optimization" (Prakash et.al. 2) of Supply Chain Networks, focusing not only on total cost but also customer service, and capacity utilization. This is particularly prescient given the number of high-profile supply chain issues that were endured throughout the low-points of the COVID-19 pandemic, which continue to plague supply chains. The authors present the utilization of a Branch-and-Bound technique to solve this problem to optimality.

In their paper studying inventory cost and fill rates, KyoungJong Park and Gyouhyung Kyung propose that particle swarm optimization is a suitable method to optimize a supply chain for these metrics, this method automatically adjusts initial inventory levels of all tiers by considering  information quality level, determined by the degree of available historic lead time data. Under this framework, tiers retain the order history of the downstream. The simulated data that they utilized offered up a major decrease in overall supply chain cost, saying that "without the optimization process... the mean of total inventory costs is \\$10.01MM, [while] ... with the optimzation process... the mean of total inventory costs is $4.26MM" (Park & Kyung 7).

In his book *Inventory Management in Multi-Echelon Networks*, Christopher Grob formulates his understanding of multi-echelon supply chain networks by focusing on minimization of reorder points, rather than overall costs, as we "implicitly minimize the weighted sum of the inventory positions due to the uniformity property... the weighted sum can thus be seen as the total value of stock in the distribution network" (Grob 32). He solves this problem by using over and underestimating iterative linear programs, approximating the non-linear constraints with piecewise linear functions that refine until a targeted optimality gap is reached.

## A Clear Problem Description

I am solving a 2 stage optimization problem - maximizing truck fill while meeting all demand, then minimizing the route length of those individual trucks, I will use a Mixed-Integer-Programming approach to solve for the truck fill rate, then optimizing the traveling salesman problem with a linear programming solver. 

## The Model

In [1]:
import numpy.random as npr
import random as rd
from ortools.linear_solver import pywraplp as glp
import lptools as lpt
import itertools

# Create a Maximization model for truck fill rate

In [70]:
#Create LP model object
mymodel = glp.Solver('Maximize Truck Fill', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Here we create a model object in GLOP, a part of Google's OR-Tools Optimization library, we'll be using Mixed Integer Programming to calculate the number of pallets that can be fit into a truck to fill it to capacity.

In [71]:
inf = mymodel.infinity()

            # name: (type 'c'=continuous or 'i'=integer, lower bound, upper bound, objective coefficient)
variables = {'P1':    ('i', 0, 1, 12), # each time we load a pallet of product 1 we get 12 units onto a truck
             'P2':    ('i', 0, 1, 9),
             'P3':    ('i', 0, 1, 17),
             'P4':    ('i', 0, 1, 32),
             'P5':    ('i', 0, 1, 30)} # the number of trucks doesn't directly impact the amount of inventory

            # name: (lower bound, upper bound, coefficient list)
constraints = {'capacity': (-inf, 100, [12, 9, 17, 32,30])} # we have 100 units of capacity per truck, co
               

mymodel.Objective().SetMaximization()

I created 4 variables corresponding to 4 products, defined as integers representing a pallet of those 4 products, each of which contributes a certain "size" (amount of product) to the overall objective function. Below this I defined a constraint on the capacity of a truck.

In [72]:
for v in variables:
    (t,lb,ub,c) = variables[v]
    if t == 'c':
        var = mymodel.NumVar(lb,ub,v)
    elif t == 'i':
        var = mymodel.IntVar(lb,ub,v)
    else:
        print('Invalid variable type =', t)
    mymodel.Objective().SetCoefficient(var,c)

In [73]:
for c in constraints:
    (lb,ub,coeff_lst) = constraints[c]
    constr = mymodel.Constraint(lb,ub,c)
    for (v,coeff) in zip(mymodel.variables(),coeff_lst):
        constr.SetCoefficient(v,coeff)

In [74]:
lpt.print_model(mymodel)

Variables:
P1, P2, P3, P4, P5 

maximize: 12.0*P1 + 9.0*P2 + 17.0*P3 + 32.0*P4 + 30.0*P5 

Subject To:
capacity: 12.0*P1 + 9.0*P2 + 17.0*P3 + 32.0*P4 + 30.0*P5 <= 100.0

Bounds:
0.0 <= P1 <= 1.0
0.0 <= P2 <= 1.0
0.0 <= P3 <= 1.0
0.0 <= P4 <= 1.0
0.0 <= P5 <= 1.0


You can see the mathematical formulation yielded by this model, maximizing the sum product of the products and their pallet sizes inside of the truck, with each producted being bounded between 0 and 1.

In [75]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 100.00
P1 = 1.00
P2 = 1.00
P3 = 1.00
P4 = 1.00
P5 = 1.00


The results yielded below indicate that an optimally filled truck will contain 1 of each of our products. In this case, each product is only allowed 1 pallet, which could be relaxed to offer greater flexibility to the model.

# Create some random demands for each of our facilities

In [76]:
# create a variable number of facilities assigned a character name
facilities = 15
facility_lst = list(map(chr, range(97, 97+facilities)))

In [84]:
trials = list(range(0,101))
dmd_lst = list()
for i in trials:
    dmd = rd.normalvariate(30,20)
    dmd_lst.append(dmd)
print(dmd_lst)

[42.65790116471774, 47.84931368388318, 34.41799775284363, 4.659653230768939, 13.570015135457815, 28.377694760418148, 48.1359740729676, 24.087918285024777, 7.657880572775571, 36.34451974331098, -10.808350246551313, 22.917878430897773, 56.57184913354922, 26.284791710640228, 53.9503596597286, 35.242772015392305, 24.055505803859383, 43.80875020780759, 4.810289397454973, 33.882622847459764, 7.920576434393823, -1.9972429440125161, 32.643751614714425, 37.801542988508764, 24.156908859528293, 12.158571483996358, 30.566148989848802, 11.913971872976106, -26.0315243187027, 29.815948993779113, 25.154600662796973, 26.553259105318567, 23.339071057793376, 55.64499358733816, 14.430818953848627, 57.47045079912578, 29.69882206869854, 3.825001743720936, 5.222706531647795, 50.45064581551518, 2.0229099813005824, 100.21000663809816, 18.690794818565685, 39.62119360649421, 59.68406808299187, 17.89890671414173, 60.273964717539215, 19.720083314680267, 15.600127640984752, 62.623801253758394, 42.53404201922572, 17

In [90]:
p1_mean_dmd = 32
p2_mean_dmd = 12
p3_mean_dmd = 42
p4_mean_dmd = 13
p1_sd = 3
p2_sd = 5
p3_sd = 15
p4_sd = 1
p5_mean_dmd = 32
p5_sd = 4
demand_lst = list()
for f in facility_lst:
    product_lst = list()
    pl_demand = round(rd.normalvariate(p1_mean_dmd,p1_sd))
    p2_demand = round(rd.normalvariate(p2_mean_dmd,p2_sd))
    p3_demand = round(rd.normalvariate(p3_mean_dmd,p3_sd))
    p4_demand = round(rd.normalvariate(p4_mean_dmd,p4_sd))
    p5_demand = round(rd.normalvariate(p5_mean_dmd,p5_sd))
    demand_lst.append(pl_demand)
    demand_lst.append(p2_demand)
    demand_lst.append(p3_demand)
    demand_lst.append(p4_demand)
    demand_lst.append(p5_demand)
    for v in variables: 
        product_lst.append(v)
        demand = dict(zip(product_lst, demand_lst))
    print('facility demand: ', demand)

facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 'P4': 13, 'P5': 29}
facility demand:  {'P1': 37, 'P2': 3, 'P3': 45, 

Above I created demands with normally distributed random variables for each of the facilities in the network, by product, and created a dictionary of all these demands to be accessed later.

## Creating unique path's that don't traverse the same node twice

In [91]:
#print the list of facilities to verify list
print(facility_lst)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o']


In [92]:
# create a list of all paths between these facilities
all_paths = list(itertools.product(facility_lst, facility_lst))

In [93]:
# remove duplcated paths e.g. (0,1) & (1,0)
res = list({*map(tuple, map(sorted,all_paths))})

In [94]:
# define element positions within the tuples
first_tuple_elements = [r[0] for r in res]
second_tuple_elements =[r[1] for r in res]

In [95]:
# create a list of path options that doesn't include staying on the same node
path_options = list()
for i in range(0,len(first_tuple_elements)):
    if first_tuple_elements[i] != second_tuple_elements[i]:
        path_options.append(res[i])

In [96]:
# print all remaining path options - should be completed
print(path_options)

[('h', 'k'), ('e', 'g'), ('g', 'i'), ('k', 'm'), ('d', 'f'), ('a', 'b'), ('d', 'n'), ('i', 'o'), ('g', 'o'), ('b', 'j'), ('f', 'g'), ('a', 'n'), ('k', 'o'), ('j', 'n'), ('c', 'n'), ('m', 'n'), ('e', 'f'), ('h', 'l'), ('i', 'k'), ('g', 'k'), ('e', 'n'), ('d', 'j'), ('a', 'g'), ('c', 'g'), ('l', 'm'), ('f', 'n'), ('b', 'e'), ('e', 'j'), ('i', 'l'), ('b', 'k'), ('g', 'l'), ('h', 'n'), ('l', 'o'), ('e', 'm'), ('b', 'h'), ('a', 'f'), ('k', 'l'), ('c', 'f'), ('b', 'm'), ('f', 'j'), ('d', 'e'), ('b', 'i'), ('d', 'k'), ('f', 'm'), ('d', 'h'), ('d', 'm'), ('b', 'o'), ('i', 'n'), ('g', 'n'), ('a', 'j'), ('a', 'h'), ('d', 'i'), ('e', 'k'), ('c', 'j'), ('c', 'h'), ('k', 'n'), ('a', 'm'), ('e', 'h'), ('j', 'm'), ('c', 'm'), ('d', 'o'), ('b', 'c'), ('f', 'k'), ('e', 'i'), ('f', 'h'), ('a', 'o'), ('j', 'o'), ('c', 'o'), ('e', 'l'), ('m', 'o'), ('e', 'o'), ('a', 'e'), ('f', 'i'), ('b', 'd'), ('b', 'l'), ('h', 'j'), ('a', 'k'), ('c', 'e'), ('j', 'k'), ('b', 'g'), ('c', 'k'), ('l', 'n'), ('h', 'm'), ('f

# Create Random Distances 

In [97]:
# create all the distances for the various paths - normally distributed random variables
distance_mean = 20
distance_std = 5
distance_lst = list()
for p in path_options: 
    distance = round(rd.normalvariate(distance_mean, distance_std),2)
    distance_final = round(distance + rd.normalvariate(200,50),2)
    distance_lst.append(distance_final)

## Integrate our distances and stops to get a shortest path problem

In [98]:
# update for our model
edge = {}

def addPath(path, cost):
    if path in edge:
        print('path already in model')
    else:
        edge[path] = cost

for p in path_options:
    addPath(p,None)

In [99]:
edge = dict(zip(edge, distance_lst))

print(edge)
#Create LP model object
mymodel = glp.Solver('ShortestPath',glp.Solver.GLOP_LINEAR_PROGRAMMING)
mymodel.Objective().SetMinimization()    # minimize total transportation cost

{('h', 'k'): 267.86, ('e', 'g'): 241.61, ('g', 'i'): 175.48, ('k', 'm'): 242.68, ('d', 'f'): 150.69, ('a', 'b'): 254.45, ('d', 'n'): 285.91, ('i', 'o'): 329.22, ('g', 'o'): 232.67, ('b', 'j'): 155.69, ('f', 'g'): 245.26, ('a', 'n'): 246.09, ('k', 'o'): 278.46, ('j', 'n'): 159.53, ('c', 'n'): 178.49, ('m', 'n'): 206.8, ('e', 'f'): 264.21, ('h', 'l'): 214.98, ('i', 'k'): 194.96, ('g', 'k'): 221.91, ('e', 'n'): 249.25, ('d', 'j'): 197.55, ('a', 'g'): 136.46, ('c', 'g'): 206.47, ('l', 'm'): 219.97, ('f', 'n'): 150.21, ('b', 'e'): 141.67, ('e', 'j'): 184.06, ('i', 'l'): 246.88, ('b', 'k'): 249.47, ('g', 'l'): 250.29, ('h', 'n'): 162.09, ('l', 'o'): 216.65, ('e', 'm'): 274.06, ('b', 'h'): 210.36, ('a', 'f'): 224.48, ('k', 'l'): 246.27, ('c', 'f'): 147.09, ('b', 'm'): 224.85, ('f', 'j'): 265.43, ('d', 'e'): 132.37, ('b', 'i'): 215.14, ('d', 'k'): 194.87, ('f', 'm'): 264.07, ('d', 'h'): 273.28, ('d', 'm'): 298.23, ('b', 'o'): 233.49, ('i', 'n'): 228.87, ('g', 'n'): 233.81, ('a', 'j'): 129.26, 

Above I combine the dictionary of the individual nodes with the distance associated with that node, creating a comprehensive dictionary of the possible edges in our optimization space.

In [41]:
node = {}

def addNode(facility, role):
    if facility in node:
        print('facility already a node')
    else:
        node[facility] = role

for f in facility_lst:
    addNode(f,None)
    
role_lst = [0] * len(facility_lst)

In [42]:
# create a list of nodes and assign them roles, -1 is the start, 1 is the finish, 0's are intermediate

role_lst[0] = -1
role_lst[len(role_lst)-1] = 1
node = dict(zip(node, role_lst))
print(node)

{'a': -1, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 1}


Here I create a beginning and end point for the shortest path model, with "a" being the starting point (facility a), and "o" being the end point.

In [43]:
# create a constraint dictionary containing a constraint for each node
constr = dict()
for n in node:
    b = node[n]
    constr[n] = mymodel.Constraint(b,b,str(n))

In [44]:
# create a variable dictionary containing a variable for each edge
# add each variable to the objective and its corresponding constraints
inf = mymodel.infinity()
path = dict()
for e in edge:
    (o,d) = e
    c = edge[e]
    path[e] = mymodel.NumVar(0, inf, str(o) + '_' + str(d))
    mymodel.Objective().SetCoefficient(path[e],c)
    constr[o].SetCoefficient(path[e],-1)
    constr[d].SetCoefficient(path[e],1)

lpt.print_model(mymodel)

Variables:
h_k, e_g, g_i, k_m, d_f, a_b, d_n, i_o, g_o, b_j, f_g, a_n, k_o, j_n, c_n, m_n, e_f, h_l, i_k, g_k, e_n, d_j, a_g, c_g, l_m, f_n, b_e, e_j, i_l, b_k, g_l, h_n, l_o, e_m, b_h, a_f, k_l, c_f, b_m, f_j, d_e, b_i, d_k, f_m, d_h, d_m, b_o, i_n, g_n, a_j, a_h, d_i, e_k, c_j, c_h, k_n, a_m, e_h, j_m, c_m, d_o, b_c, f_k, e_i, f_h, a_o, j_o, c_o, e_l, m_o, e_o, a_e, f_i, b_d, b_l, h_j, a_k, c_e, j_k, b_g, c_k, l_n, h_m, f_l, a_c, f_o, h_i, d_l, a_i, d_g, c_i, n_o, h_o, a_d, a_l, i_j, g_j, b_f, g_h, j_l, b_n, c_d, c_l, i_m, g_m 

minimize: 183.64*h_k + 185.33*e_g + 245.17*g_i + 220.34*k_m + 187.8*d_f + 167.28*a_b + 162.18*d_n + 311.45*i_o + 228.96*g_o + 227.01*b_j + 229.66*f_g + 249.09*a_n + 294.83*k_o + 175.42*j_n + 164.37*c_n + 164.57*m_n + 145.46*e_f + 228.59*h_l + 177.4*i_k + 241.65*g_k + 217.55*e_n + 328.04*d_j + 148.33*a_g + 276.01*c_g + 229.86*l_m + 224.51*f_n + 133.88*b_e + 175.85*e_j + 234.5*i_l + 249.7*b_k + 261.14*g_l + 182.96*h_n + 210.34*l_o + 252.41*e_m + 107.65*b_h + 20

In [45]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 177.00
a_o = 1.00


Above we see the shortest path between the two nodes was a straight line, without any intermediate nodes.

## Results/Conclusions

There are several key failings to this model - firstly being it's disjointness. There is poor transfer of information between the stages of the optimization, a truck fill is found, then a shortest path is found, but that shortest path is not guaranteed with a certificate of optimality to be the optimal path for the optimally filled truck, given our demand. Further, this formulation is not achieving the goal of a travelling salesperson model, but rather a shortest path. Integrating these two key improvements could make this a very exciting model, as integrating an understanding of optimal trucks into our planning for optimal paths would be beneficial to our supply chain costs overall. Quantifying our inventory costs in this situation would also be key. What we can take away from this model, however, is that MILP is an appropriate approach towards developing truck fullness heuristics to inform supply chain decisions, and that we can take the methodical approach outlined in the above simulation to develop a dataset to optimize a network over. 

# Works Cited

Park, KyoungJong, and Gyouhyung Kyung. “Optimization of Total Inventory Cost and Order Fill Rate in a Supply Chain Using PSO.” The International Journal of Advanced Manufacturing Technology, vol. 70, no. 9-12, 2013, pp. 1533–1541., https://doi.org/10.1007/s00170-013-5399-6. 

Prakash, A., et al. “Network Optimization in Supply Chain: A KBGA Approach.” Decision Support Systems, vol. 52, no. 2, 2012, pp. 528–538., https://doi.org/10.1016/j.dss.2011.10.024. 

Grob, Christopher. Inventory Management in Multi-Echelon Networks on the Optimization of Reorder Points. Springer Fachmedien Wiesbaden, 2019. 